In [95]:
import pandas as pd
import tensorflow as tf
import gensim
from gensim.models import Word2Vec
from tensorflow import keras
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
import nltk
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Sai
[nltk_data]     Charan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Sai
[nltk_data]     Charan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [56]:
df = pd.read_excel(r"E:\jupyter_notebook\Datasets\Movie Performance Predictor\Movie_Performance_Data.xlsx")
df

,Movie Name,Movie Description,Movie Rating,Cast_1,Cast_2,Cast_3,Director,Budget,Collection,Verdict
0,PK,An alien on Earth loses the only device he can...,8.1,Aamir Khan,Anushka Sharma,Sanjay Dutt,Rajkumar Hirani,85.0,792.0,All Time Blockbuster
1,Happy New Year,A gang of amateur dancers takes part in a danc...,5.0,Shah Rukh Khan,Deepika Padukone,Abhishek Bachchan,Farah Khan,150.0,397.0,Hit
2,Kick,An adrenaline junkie walks away from a whirlwi...,6.0,Salman Khan,Jacqueline Fernandez,Nawazuddin Siddiqui,Sajid Nadiadwala,140.0,378.0,Super Hit
3,Bang Bang,A young bank receptionist gets mixed up with R...,5.6,Hrithik Roshan,Katrina Kaif,Pawan Malhotra,Siddharth Anand,140.0,340.0,Super Hit
4,Singham Returns,Owing to the wrongdoings affiliated with evils...,5.7,Ajay Devgn,Kareena Kapoor,Amole Gupte,Rohit Shetty,80.0,216.0,Super Hit
...,...,...,...,...,...,...,...,...,...,...
668,NTR Kathanayakudu,Based on the journey of the legendary Telugu a...,5.8,Nandamuri Balakrishna,Vidya Balan,Kaikala Satyanarayana,Radha Krishna Jagarlamudi,1.0,NaN,Hit
669,Love Action Drama,A man goes through all kinds of action to impr...,5.2,Nivin Pauly,Nayanthara,Aju Varghese,Dhyan Sreenivasan,1.0,6.0,Super Hit
670,Driving Licence,A famous actor needs to renew his driver's lic...,7.2,Prithviraj Sukumaran,Suraj Venjaramoodu,Miya George,Jean Paul Lal,7.0,1.0,Disaster
671,Mr. Local,A happy-go-lucky guy lands in trouble after he...,3.3,Sivakarthikeyan,Nayanthara,Radhika Sarathkumar,M. Rajesh,3.0,4.0,Blockbuster


In [80]:
df.isna().sum()

Movie Name           0
Movie Description    0
Movie Rating         0
Cast_1               0
Cast_2               0
Cast_3               0
Director             0
Budget               0
Collection           0
Verdict              0
dtype: int64

In [57]:
df = df.dropna()

In [73]:
x = df['Movie Description']
y = df.Budget

In [74]:
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=42)

In [79]:
print(x_train.dtypes)
print(y_train.dtypes)

object
float64


In [75]:
x_train

80     Arjun Reddy, a short-tempered house surgeon, g...
602    A daring robbery in a jewellery showroom kicks...
443    After the events of Stree, the town of Chander...
540    Vijay Rajendran is a happy to-go lucky man. Th...
504    The lives of two brothers from a humble backgr...
                             ...                        
100    A detective is hungry for a challenging case t...
139    Anand and Anasuya meet each other on a train r...
371    After being conned by the woman he loves, a fi...
536    When his friends in Punjab struggle to clear t...
135    Following the attack on the Indian Embassy in ...
Name: Movie Description, Length: 389, dtype: object

In [76]:
print(x_test.dtype)  # Should be 'object' or 'string'
print(x_test.head()) 

object
376    A tough police officer sets out to track down ...
125    An intense battle breaks out between an evil s...
6      A military officer attempts to hunt down a ter...
200    Bharat, a young graduate who becomes the C.M o...
122    When a small village near a forest is under th...
Name: Movie Description, dtype: object


In [136]:
vectoriser = TfidfVectorizer(max_features=100000)
x_train_vec = vectoriser.fit_transform(x_train)
x_test_vec = vectoriser.transform(x_test)
x_train_vec = x_train_vec.toarray() 
x_test_vec = x_test_vec.toarray()

In [130]:
scaler = MinMaxScaler()
y_train_scaled = y_train/max(y)
y_test_scaled = y_test/max(y)

In [131]:
max(y)

600.0

In [104]:
optimiser = Adam(learning_rate=0.01)

In [139]:
model = keras.Sequential([
    keras.layers.Dense(512, activation='relu',kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
model.fit(x_train_vec, y_train_scaled, epochs=100)

Epoch 1/100
13/13 [==============================] - 4s 35ms/step - loss: 6.6532 - mean_absolute_error: 0.4331
Epoch 2/100
13/13 [==============================] - 0s 32ms/step - loss: 3.7798 - mean_absolute_error: 0.4183
Epoch 3/100
13/13 [==============================] - 0s 33ms/step - loss: 2.6839 - mean_absolute_error: 0.3919
Epoch 4/100
13/13 [==============================] - 0s 33ms/step - loss: 2.2001 - mean_absolute_error: 0.3707
Epoch 5/100
13/13 [==============================] - 0s 33ms/step - loss: 1.9004 - mean_absolute_error: 0.3495
Epoch 6/100
13/13 [==============================] - 0s 33ms/step - loss: 1.6648 - mean_absolute_error: 0.3346
Epoch 7/100
13/13 [==============================] - 0s 34ms/step - loss: 1.4570 - mean_absolute_error: 0.3088
Epoch 8/100
13/13 [==============================] - 0s 32ms/step - loss: 1.2944 - mean_absolute_error: 0.3008
Epoch 9/100
13/13 [==============================] - 0s 33ms/step - loss: 1.1560 - mean_absolute_error: 0.2793
E

13/13 [==============================] - 0s 37ms/step - loss: 0.1948 - mean_absolute_error: 0.0576
Epoch 75/100
13/13 [==============================] - 0s 38ms/step - loss: 0.1841 - mean_absolute_error: 0.0588
Epoch 76/100
13/13 [==============================] - 0s 36ms/step - loss: 0.1854 - mean_absolute_error: 0.0588
Epoch 77/100
13/13 [==============================] - 0s 36ms/step - loss: 0.1916 - mean_absolute_error: 0.0621
Epoch 78/100
13/13 [==============================] - 0s 36ms/step - loss: 0.1914 - mean_absolute_error: 0.0621
Epoch 79/100
13/13 [==============================] - 0s 35ms/step - loss: 0.1909 - mean_absolute_error: 0.0604
Epoch 80/100
13/13 [==============================] - 0s 36ms/step - loss: 0.1871 - mean_absolute_error: 0.0578
Epoch 81/100
13/13 [==============================] - 0s 36ms/step - loss: 0.1889 - mean_absolute_error: 0.0602
Epoch 82/100
13/13 [==============================] - 0s 34ms/step - loss: 0.1782 - mean_absolute_error: 0.0641
Epoch

In [140]:
model.evaluate(x_test_vec, y_test_scaled)

5/5 [==============================] - 0s 5ms/step - loss: 0.1634 - mean_absolute_error: 0.0592


[0.16339154541492462, 0.05919862166047096]

In [141]:
print(type(x_test_vec))  # Should be a numpy array or list
print(x_test_vec.dtype)  # Should be float32 or float64

<class 'numpy.ndarray'>
float64


In [142]:
y_pred = model.predict(x_test_vec)
y_pred_org = y_pred * max(y)
y_test_org = y_test * max(y)

5/5 [==============================] - 0s 4ms/step


In [150]:
y_pred_org[:10]

array([[32.45253 ],
       [42.6081  ],
       [13.538986],
       [26.8942  ],
       [33.275017],
       [21.886946],
       [20.593136],
       [15.936045],
       [27.63697 ],
       [18.815783]], dtype=float32)

In [151]:
y_test[:10]

376    23.0
125    50.0
6      70.0
200    70.0
122    25.0
653     4.0
463    30.0
324     1.0
225    10.0
583     1.0
Name: Budget, dtype: float64